In [1]:
import manim as mn
from manim import *

config.media_width = "100%"
config.verbosity = "WARNING"

print(mn.__version__)

0.19.0


In [2]:
%%manim -qm RobotArmKinematics

from manim import *
import numpy as np

class RobotArmKinematics(ThreeDScene):
    """
    3軸ロボットアームの順運動学・逆運動学を説明するアニメーション
    """
    
    # アームのリンク長
    LINK_1_LENGTH = 1.5
    LINK_2_LENGTH = 1.2
    LINK_3_LENGTH = 1.0
    
    def construct(self):
        """メインメソッド：アニメーションのシーケンスを制御"""
        # カメラをトップビューに設定（上から見下ろす視点）
        self.set_camera_orientation(phi=0 * DEGREES, theta=-90 * DEGREES, zoom=0.8)
        
        # 基本的なシーン設定
        self.setup_scene()
        
        # 各セクションの実行
        self.show_introduction()
        self.demonstrate_forward_kinematics()
        self.demonstrate_inverse_kinematics()
        self.compare_kinematics()
        self.show_conclusion()
    
    def setup_scene(self):
        """シーンの基本要素（座標系など）をセットアップ"""
        # 座標系の作成
        self.axes = ThreeDAxes(
            x_range=[-5, 5, 1],
            y_range=[-5, 5, 1],
            z_range=[-1, 5, 1],
            x_length=10,
            y_length=10,
            z_length=6,
        ).scale(0.7)
        
        # 軸ラベル - カメラに固定
        x_label = Text("x", font="IPAGothic").scale(0.8)
        y_label = Text("y", font="IPAGothic").scale(0.8)
        z_label = Text("z", font="IPAGothic").scale(0.8)
        
        # 軸ラベルをカメラに固定し、位置を調整
        self.add_fixed_in_frame_mobjects(x_label, y_label, z_label)
        x_label.to_corner(DOWN + RIGHT)
        y_label.to_corner(DOWN + LEFT)
        z_label.to_corner(UP + RIGHT)
        
        # 原点に三次元座標系を配置
        self.add(self.axes)
    
    def show_introduction(self):
        """イントロダクションセクション：タイトルとロボットアーム紹介"""
        # タイトル - カメラに固定
        title = Text("3軸ロボットアームの運動学", font="IPAGothic")
        subtitle = Text("順運動学と逆運動学の説明", font="IPAGothic", font_size=36)
        
        # タイトルをカメラに固定
        self.add_fixed_in_frame_mobjects(title, subtitle)
        title.to_edge(UP)
        subtitle.next_to(title, DOWN)
        
        self.play(Write(title))
        self.wait()
        self.play(FadeOut(title), FadeIn(subtitle))
        self.wait()
        
        # 初期角度でロボットアームを作成
        initial_angles = (0, PI/4, PI/4)
        self.robot_arm, end_pos, joint_positions = self.create_robot_arm(*initial_angles)
        
        self.play(Create(self.robot_arm))
        self.wait()
        
        # 関節とエンドエフェクタのラベル - カメラに固定
        labels = []
        label_texts = ["関節1", "関節2", "関節3", "エンドエフェクタ"]
        positions = [*joint_positions, end_pos]
        
        for i, (text, pos) in enumerate(zip(label_texts, positions)):
            label = Text(text, font="IPAGothic", font_size=24)
            label.scale(0.8)  # ラベルサイズを調整
            self.add_fixed_in_frame_mobjects(label)
            
            # ラベル位置をカメラビューに適応させる
            screen_pos = self.axes.c2p(*pos)
            offset = UP * 0.5 if i != 3 else RIGHT * 0.7  # エンドエフェクタは右側に配置
            label.move_to(screen_pos + offset)
            
            labels.append(label)
        
        self.play(*[Write(label) for label in labels])
        self.wait(2)
        
        # ラベルと字幕をフェードアウト
        self.play(
            *[FadeOut(label) for label in labels],
            FadeOut(subtitle)
        )
    
    def demonstrate_forward_kinematics(self):
        """順運動学のデモンストレーション"""
        # 順運動学の説明テキスト - カメラに固定
        fk_title = Text("順運動学 (Forward Kinematics)", font="IPAGothic", font_size=36)
        fk_desc = Text(
            "関節角度から先端位置を計算する方法",
            font="IPAGothic",
            font_size=24
        )
        
        self.add_fixed_in_frame_mobjects(fk_title, fk_desc)
        fk_title.to_corner(UP + LEFT)
        fk_desc.next_to(fk_title, DOWN, aligned_edge=LEFT)
        
        self.play(Write(fk_title))
        self.play(Write(fk_desc))
        self.wait()
        
        # 数式表示 - カメラに固定
        eq_group = self._create_fk_equations()
        self.add_fixed_in_frame_mobjects(eq_group)
        eq_group.to_corner(DOWN + LEFT)
        
        self.play(Write(eq_group))
        self.wait(2)
        
        # 異なる角度でアームを動かして順運動学をデモ
        angles_list = [
            (PI/4, PI/4, 0),
            (PI/2, PI/6, PI/4),
            (3*PI/4, PI/3, PI/6),
            (0, PI/2, PI/3)
        ]
        
        for angles in angles_list:
            new_arm, new_end_pos, _ = self.create_robot_arm(*angles)
            
            # 角度と位置のテキスト - カメラに固定
            angle_text = Text(
                f"θ₁={angles[0]*180/PI:.1f}°, θ₂={angles[1]*180/PI:.1f}°, θ₃={angles[2]*180/PI:.1f}°",
                font="IPAGothic",
                font_size=24
            )
            pos_text = Text(
                f"先端位置: ({new_end_pos[0]:.2f}, {new_end_pos[1]:.2f}, {new_end_pos[2]:.2f})",
                font="IPAGothic",
                font_size=24
            )
            
            self.add_fixed_in_frame_mobjects(angle_text, pos_text)
            angle_text.to_corner(DOWN + RIGHT)
            pos_text.next_to(angle_text, UP, aligned_edge=RIGHT)
            
            self.play(Transform(self.robot_arm, new_arm))
            self.play(Write(angle_text), Write(pos_text))
            self.wait(2)
            self.play(FadeOut(angle_text), FadeOut(pos_text))
        
        self.play(FadeOut(fk_title), FadeOut(fk_desc), FadeOut(eq_group))
    
    def demonstrate_inverse_kinematics(self):
        """逆運動学のデモンストレーション"""
        # 初期位置にアームをリセット
        initial_angles = (0, PI/4, PI/4)
        initial_arm, _, _ = self.create_robot_arm(*initial_angles)
        self.play(Transform(self.robot_arm, initial_arm))
        self.wait()
        
        # 逆運動学の説明 - カメラに固定
        ik_title = Text("逆運動学 (Inverse Kinematics)", font="IPAGothic", font_size=36)
        ik_desc = Text(
            "先端位置から関節角度を計算する方法",
            font="IPAGothic",
            font_size=24
        )
        
        self.add_fixed_in_frame_mobjects(ik_title, ik_desc)
        ik_title.to_corner(UP + LEFT)
        ik_desc.next_to(ik_title, DOWN, aligned_edge=LEFT)
        
        self.play(Write(ik_title))
        self.play(Write(ik_desc))
        self.wait()
        
        # 逆運動学の説明を追加 - カメラに固定
        ik_steps = self._create_ik_steps()
        self.add_fixed_in_frame_mobjects(ik_steps)
        ik_steps.to_corner(DOWN + LEFT)
        
        self.play(Write(ik_steps))
        self.wait(2)
        
        # 目標点でのデモンストレーション
        target_positions = [
            np.array([2.5, 1.5, 2.0]),  # 右上
            np.array([1.0, 2.5, 1.0]),  # 左上
            np.array([2.5, 0.0, 1.5]),  # 真右
            np.array([3.0, 1.0, 0.5])   # 右下
        ]
        
        for i, target_pos in enumerate(target_positions):
            # 目標点の表示
            target_result = self._demonstrate_ik_target(i, target_pos)
            if target_result:
                self.wait(2)
        
        self.play(FadeOut(ik_title), FadeOut(ik_desc), FadeOut(ik_steps))
    
    def compare_kinematics(self):
        """順運動学と逆運動学の比較セクション"""
        # 初期位置にリセット
        initial_angles = (0, PI/4, PI/4)
        initial_arm, _, _ = self.create_robot_arm(*initial_angles)
        self.play(Transform(self.robot_arm, initial_arm))
        
        # 比較表のタイトル - カメラに固定
        comparison_title = Text("順運動学 vs 逆運動学", font="IPAGothic", font_size=36)
        self.add_fixed_in_frame_mobjects(comparison_title)
        comparison_title.to_edge(UP)
        
        self.play(Write(comparison_title))
        
        # 比較表の作成 - カメラに固定
        comparison_table = self._create_comparison_table()
        self.add_fixed_in_frame_mobjects(comparison_table)
        comparison_table.next_to(comparison_title, DOWN, buff=0.5)
        
        self.play(Write(comparison_table), run_time=3)
        self.wait(3)
        
        self.play(FadeOut(comparison_title), FadeOut(comparison_table))
    
    def show_conclusion(self):
        """まとめセクション"""
        # ロボットアームのフェードアウト
        self.play(FadeOut(self.robot_arm))
        
        # まとめのテキスト - カメラに固定
        conclusion_title = Text("まとめ", font="IPAGothic", font_size=48)
        conclusion_points = self._create_conclusion_points()
        
        self.add_fixed_in_frame_mobjects(conclusion_title, conclusion_points)
        conclusion_title.to_edge(UP)
        conclusion_points.next_to(conclusion_title, DOWN, buff=0.5)
        
        self.play(Write(conclusion_title))
        self.play(Write(conclusion_points))
        self.wait(3)
        
        self.play(FadeOut(conclusion_title), FadeOut(conclusion_points))
        
        # 終了メッセージ - カメラに固定
        final_text = Text("ご視聴ありがとうございました", font="IPAGothic", font_size=48)
        self.add_fixed_in_frame_mobjects(final_text)
        final_text.move_to(ORIGIN)
        
        self.play(Write(final_text))
        self.wait(2)
        self.play(FadeOut(final_text))
    
    def create_robot_arm(self, theta1, theta2, theta3):
        """
        指定された角度でロボットアームを作成
        
        Parameters:
            theta1 (float): 第1関節の角度（ラジアン）
            theta2 (float): 第2関節の角度（ラジアン）
            theta3 (float): 第3関節の角度（ラジアン）
            
        Returns:
            tuple: (ロボットアーム全体のVGroup, エンドエフェクタ位置, 関節位置のタプル)
        """
        # リンク長の取得
        l1, l2, l3 = self.LINK_1_LENGTH, self.LINK_2_LENGTH, self.LINK_3_LENGTH
        
        # 順運動学を使って関節位置を計算
        # ベース位置
        j1_pos = np.array([0, 0, 0.1])
        
        # 関節2の位置（theta1回転後）
        x1 = l1 * np.cos(theta1)
        y1 = l1 * np.sin(theta1)
        z1 = 0.1
        j2_pos = np.array([x1, y1, z1])
        
        # 関節3の位置（theta2回転後）
        x2 = x1 + l2 * np.cos(theta1) * np.cos(theta2)
        y2 = y1 + l2 * np.sin(theta1) * np.cos(theta2)
        z2 = z1 + l2 * np.sin(theta2)
        j3_pos = np.array([x2, y2, z2])
        
        # エンドエフェクタの位置（theta3回転後）
        x3 = x2 + l3 * np.cos(theta1) * np.cos(theta2 + theta3)
        y3 = y2 + l3 * np.sin(theta1) * np.cos(theta2 + theta3)
        z3 = z2 + l3 * np.sin(theta2 + theta3)
        end_pos = np.array([x3, y3, z3])
        
        # ビジュアル要素の作成
        # ベース
        base = Cylinder(radius=0.3, height=0.2, fill_color=BLUE, fill_opacity=0.8)
        base.move_to(self.axes.c2p(0, 0, 0))
        
        # 関節
        joint1 = Sphere(radius=0.15, fill_color=RED, fill_opacity=0.8)
        joint1.move_to(self.axes.c2p(*j1_pos))
        
        joint2 = Sphere(radius=0.15, fill_color=RED, fill_opacity=0.8)
        joint2.move_to(self.axes.c2p(*j2_pos))
        
        joint3 = Sphere(radius=0.15, fill_color=RED, fill_opacity=0.8)
        joint3.move_to(self.axes.c2p(*j3_pos))
        
        end_effector = Sphere(radius=0.1, fill_color=GREEN, fill_opacity=0.8)
        end_effector.move_to(self.axes.c2p(*end_pos))
        
        # リンク
        link1 = Line3D(
            start=self.axes.c2p(*j1_pos),
            end=self.axes.c2p(*j2_pos),
            thickness=0.05,
            color=WHITE
        )
        
        link2 = Line3D(
            start=self.axes.c2p(*j2_pos),
            end=self.axes.c2p(*j3_pos),
            thickness=0.05,
            color=WHITE
        )
        
        link3 = Line3D(
            start=self.axes.c2p(*j3_pos),
            end=self.axes.c2p(*end_pos),
            thickness=0.05,
            color=WHITE
        )
        
        # すべてをグループ化
        arm = VGroup(base, joint1, joint2, joint3, end_effector, link1, link2, link3)
        
        return arm, end_pos, (j1_pos, j2_pos, j3_pos)
    
    def inverse_kinematics(self, target_pos):
        """
        目標位置から関節角度を計算（逆運動学）
        
        Parameters:
            target_pos (np.array): 目標位置の座標 [x, y, z]
            
        Returns:
            tuple: 計算された関節角度(theta1, theta2, theta3)
        """
        l1, l2, l3 = self.LINK_1_LENGTH, self.LINK_2_LENGTH, self.LINK_3_LENGTH
        x, y, z = target_pos
        
        # ステップ1: theta1を計算 (x-y平面上での角度)
        theta1 = np.arctan2(y, x)
        
        # x-y平面上での距離
        r = np.sqrt(x**2 + y**2)
        
        # 第2リンクの開始点までの距離を計算
        r1 = l1  # 最初のリンク長がx-y平面上での距離
        
        # 残りの距離 (target点までの距離からr1を引く)
        r2 = r - r1
        
        # x-y平面からの高さ
        height = z
        
        # リンク2と3の長さを使って三角形の解を求める
        D = (r2**2 + height**2 - l2**2 - l3**2) / (2 * l2 * l3)
        D = np.clip(D, -1.0, 1.0)  # 数値誤差に対処
        
        # ステップ2: theta3を計算 (第3関節の角度)
        theta3 = np.arccos(D)
        
        # ステップ3: theta2を計算 (第2関節の角度)
        theta2 = np.arctan2(height, r2) - np.arctan2(l3 * np.sin(theta3), l2 + l3 * np.cos(theta3))
        
        return theta1, theta2, theta3
    
    def _create_fk_equations(self):
        """順運動学の数式を作成"""
        l_values = Text(
            "アームの各リンクの長さ: l₁=1.5, l₂=1.2, l₃=1.0",
            font="IPAGothic",
            font_size=24
        )
        
        fk_eq1 = MathTex(
            r"x_{end} = l_1\cos\theta_1 + l_2\cos\theta_1\cos\theta_2 + l_3\cos\theta_1\cos(\theta_2 + \theta_3)"
        )
        fk_eq1.scale(0.7)
        
        fk_eq2 = MathTex(
            r"y_{end} = l_1\sin\theta_1 + l_2\sin\theta_1\cos\theta_2 + l_3\sin\theta_1\cos(\theta_2 + \theta_3)"
        )
        fk_eq2.scale(0.7)
        
        fk_eq3 = MathTex(
            r"z_{end} = l_2\sin\theta_2 + l_3\sin(\theta_2 + \theta_3)"
        )
        fk_eq3.scale(0.7)
        
        equations = VGroup(l_values, fk_eq1, fk_eq2, fk_eq3).arrange(DOWN, aligned_edge=LEFT)
        return equations
    
    def _create_ik_steps(self):
        """逆運動学の手順を説明するテキストを作成"""
        ik_eq_text = Text(
            "逆運動学では以下の手順で解を求めます:",
            font="IPAGothic",
            font_size=24
        )
        
        ik_step1 = Text(
            "1. θ₁ = atan2(y, x)  (ベースの回転角)",
            font="IPAGothic",
            font_size=24
        )
        
        ik_step2 = Text(
            "2. 幾何学と三角法を使って θ₂ と θ₃ を計算",
            font="IPAGothic",
            font_size=24
        )
        
        ik_steps = VGroup(ik_eq_text, ik_step1, ik_step2).arrange(DOWN, aligned_edge=LEFT)
        return ik_steps
    
    def _demonstrate_ik_target(self, index, target_pos):
        """
        指定した目標位置に対する逆運動学のデモンストレーション
        
        Parameters:
            index (int): 目標点のインデックス
            target_pos (np.array): 目標座標
            
        Returns:
            bool: 成功したかどうか
        """
        # 目標点を表示
        target_dot = Sphere(radius=0.12, fill_color=YELLOW, fill_opacity=0.8)
        target_dot.move_to(self.axes.c2p(*target_pos))
        
        # テキスト表示 - カメラに固定
        target_text = Text(
            f"目標点 {index+1}: ({target_pos[0]:.1f}, {target_pos[1]:.1f}, {target_pos[2]:.1f})",
            font="IPAGothic",
            font_size=24
        )
        self.add_fixed_in_frame_mobjects(target_text)
        target_text.to_edge(RIGHT, buff=0.5)
        
        self.add(target_dot)
        self.play(Write(target_text))
        
        # 逆運動学を計算して関節角度を取得
        try:
            theta1, theta2, theta3 = self.inverse_kinematics(target_pos)
            
            # 計算結果を表示 - カメラに固定
            ik_result_text = Text(
                f"計算された関節角度:\nθ₁={theta1*180/PI:.1f}°\nθ₂={theta2*180/PI:.1f}°\nθ₃={theta3*180/PI:.1f}°",
                font="IPAGothic",
                font_size=24
            )
            self.add_fixed_in_frame_mobjects(ik_result_text)
            ik_result_text.next_to(target_text, DOWN, aligned_edge=LEFT)
            
            self.play(Write(ik_result_text))
            
            # 関節角度からロボットアームを更新
            new_arm, end_pos, _ = self.create_robot_arm(theta1, theta2, theta3)
            self.play(Transform(self.robot_arm, new_arm))
            
            # 実際の到達点と目標点の差を表示 - カメラに固定
            error_text = Text(
                f"誤差: {np.linalg.norm(end_pos - target_pos):.3f} 単位",
                font="IPAGothic",
                font_size=24
            )
            self.add_fixed_in_frame_mobjects(error_text)
            error_text.next_to(ik_result_text, DOWN, aligned_edge=LEFT)
            
            self.play(Write(error_text))
            
            # クリーンアップ
            self.play(
                FadeOut(target_text),
                FadeOut(ik_result_text),
                FadeOut(error_text)
            )
            
            succeeded = True
            
        except Exception as e:
            # エラーメッセージ表示 - カメラに固定
            error_msg = Text(
                "この位置はロボットの可動範囲外です",
                font="IPAGothic",
                font_size=24,
                color=RED
            )
            self.add_fixed_in_frame_mobjects(error_msg)
            error_msg.next_to(target_text, DOWN, aligned_edge=LEFT)
            
            self.play(Write(error_msg))
            
            # クリーンアップ
            self.play(
                FadeOut(target_text),
                FadeOut(error_msg)
            )
            
            succeeded = False
        
        self.remove(target_dot)
        return succeeded
    
    def _create_comparison_table(self):
        """順運動学と逆運動学の比較表を作成"""
        # ヘッダー
        header_fk = Text("順運動学", font="IPAGothic", font_size=30)
        header_ik = Text("逆運動学", font="IPAGothic", font_size=30)
        
        row_headers = VGroup(header_fk, header_ik).arrange(RIGHT, buff=2.5)
        
        # 入力と出力の行
        fk_input = Text("入力: 関節角度 (θ₁,θ₂,θ₃)", font="IPAGothic", font_size=24)
        fk_output = Text("出力: エンドエフェクタ位置 (x,y,z)", font="IPAGothic", font_size=24)
        
        ik_input = Text("入力: エンドエフェクタ位置 (x,y,z)", font="IPAGothic", font_size=24)
        ik_output = Text("出力: 関節角度 (θ₁,θ₂,θ₃)", font="IPAGothic", font_size=24)
        
        fk_col = VGroup(fk_input, fk_output).arrange(DOWN, aligned_edge=LEFT)
        ik_col = VGroup(ik_input, ik_output).arrange(DOWN, aligned_edge=LEFT)
        
        input_output_row = VGroup(fk_col, ik_col).arrange(RIGHT, buff=1.0, aligned_edge=UP)
        input_output_row.next_to(row_headers, DOWN, buff=0.5, aligned_edge=LEFT)
        
        # 計算の複雑さの行
        fk_complexity = Text("計算: 単純、一意的", font="IPAGothic", font_size=24)
        ik_complexity = Text("計算: 複雑、複数解の可能性", font="IPAGothic", font_size=24)
        
        complexity_row = VGroup(fk_complexity, ik_complexity).arrange(RIGHT, buff=1.5)
        complexity_row.next_to(input_output_row, DOWN, buff=0.5, aligned_edge=LEFT)
        
        # すべてをグループ化
        comparison_table = VGroup(row_headers, input_output_row, complexity_row)
        
        return comparison_table
    
    def _create_conclusion_points(self):
        """まとめポイントを作成"""
        conclusion_points = VGroup(
            Text("1. 順運動学: 関節角度から先端位置を容易に計算できる", font="IPAGothic", font_size=30),
            Text("2. 逆運動学: 目標位置から必要な関節角度を導出", font="IPAGothic", font_size=30),
            Text("3. ロボット制御では両方が重要", font="IPAGothic", font_size=30),
            Text("4. 実際のロボットでは、物理的制約や特異点にも注意", font="IPAGothic", font_size=30)
        ).arrange(DOWN, aligned_edge=LEFT, buff=0.5)
        
        return conclusion_points

Manim Community v0.19.0